In [ ]:
%load_ext autoreload
%autoreload 2

Get the usual import

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Chose your args


In [ ]:
#* Chose his args 

class Args:
    def __init__(self):
        # basic config
        self.task_name = 'long_term_forecast'
        self.is_training = 0
        self.model_id = 'TEST'
        self.model = 'FEDformer'
        self.num_itr=0
        # data loader
        self.data = 'NTU'
        self.root_path = './dataset/NTU_RGB+D/'
        self.data_path = 'numpyed/'
        self.features = 'M'
        self.target = 'OT'
        self.freq = 'h'
        self.checkpoints = './checkpoints/'

        # forecasting task
        self.seq_len = 16
        self.label_len = 30
        self.pred_len = 35
        self.seasonal_patterns = 'Monthly'

        # inputation task
        self.mask_rate = 0.25

        # anomaly detection task
        self.anomaly_ratio = 0.25

        # model define
        self.top_k = 5
        self.num_kernels = 6
        self.enc_in = 75
        self.dec_in = 75
        self.c_out = 75
        self.d_model = 512
        self.n_heads = 8
        self.e_layers = 2
        self.d_layers = 1
        self.d_ff = 2048
        self.moving_avg = 25
        self.factor = 3
        self.distil = True
        self.dropout = 0.1
        self.embed = 'timeNTU'
        self.activation = 'gelu'
        self.output_attention = False

        # optimization
        self.num_workers = 10
        self.itr = 1
        self.train_epochs = 14
        self.batch_size = 256
        self.patience = 3
        self.learning_rate = 10**(-3)
        self.des = 'Exp'
        self.loss = 'MSE'
        self.lradj = 'sem_constant'
        self.use_amp = False

        # GPU
        self.use_gpu = False
        self.gpu = 0
        self.use_multi_gpu = False
        self.devices = '0'

        # de-stationary projector params
        self.p_hidden_dims = [128, 128]
        self.p_hidden_layers = 2

        # NTU_RG
        self.get_time_value = 1
        self.get_cat_value = 0
        self.ii=0
        self.preprocess=1
        self.split_train_test="action"
        self.num_itr=0
        self.augment=False
        self.start_checkpoint=False
args=Args()
from utils.constantes import get_settings
setting=get_settings(args)

In [ ]:
#*  args used for real script
from utils.constantes import get_settings
class Args:
    def __init__(self):
        # basic config
        self.task_name = 'long_term_forecast'
        self.is_training = 0
        self.model_id = 'Usable'
        self.model = 'FEDformer'
        self.num_itr=0
        # data loader
        self.data = 'NTU'
        self.root_path = './dataset/NTU_RGB+D/'
        self.data_path = 'numpyed/'
        self.features = 'M'
        self.target = 'OT'
        self.freq = 'h'
        self.checkpoints = './checkpoints/'

        # forecasting task
        self.seq_len = 16
        self.label_len = 32
        self.pred_len = 32
        self.seasonal_patterns = 'Monthly'

        # inputation task
        self.mask_rate = 0.25

        # anomaly detection task
        self.anomaly_ratio = 0.25

        # model define
        self.top_k = 5
        self.num_kernels = 6
        self.enc_in = 75
        self.dec_in = 75
        self.c_out = 75
        self.d_model = 512
        self.n_heads = 8
        self.e_layers = 2
        self.d_layers = 1
        self.d_ff = 2048
        self.moving_avg = 25
        self.factor = 3
        self.distil = True
        self.dropout = 0.1
        self.embed = 'timeNTU'
        self.activation = 'gelu'
        self.output_attention = False

        # optimization
        self.num_workers = 10
        self.itr = 1
        self.train_epochs = 14
        self.batch_size = 256
        self.patience = 3
        self.learning_rate = 10**(-3)
        self.des = 'Exp'
        self.loss = 'MSE'
        self.lradj = 'sem_constant'
        self.use_amp = False

        # GPU
        self.use_gpu = False
        self.gpu = 0
        self.use_multi_gpu = False
        self.devices = '0'

        # de-stationary projector params
        self.p_hidden_dims = [128, 128]
        self.p_hidden_layers = 2

        # NTU_RG
        self.get_time_value = 1
        self.get_cat_value = 0
        self.ii=0
        self.preprocess=1
        self.split_train_test="action"
        self.start_checkpoint=False
        self.augment=False
        self.no_test=False
args=Args()

setting=get_settings(args)


#  Fetch data on the dataset

 - data path
 

In [ ]:
# get a data path or a root path
import os
root_path='./dataset/NTU_RGB+D/'
data_path_npy=os.path.join(root_path,'numpyed/')
data_path_skel=os.path.join(root_path,'raw/')


In [ ]:
# Path of a .skeleton
path_skeleton="dataset/NTU_RGB+D/raw/S001C001P001R001A001.skeleton"

***

### get a .csv with the most important features (Data Set preprocessing)

1. Value to test your data fast


In [ ]:
# NTU_RGB
path_summary='./dataset/NTU_RGB+D/summary_NTU/summary_NTU.csv'
path_avant_dataset='./dataset/NTU_RGB+D/summary_NTU/liste_NTU_skeleton_4.csv'
categorical=['nbodys', 'filename', 'actor', 'acti', 'camera', 'scene', 'repet','njoints']
particulier=['nb_frames', 'debut_frame', 'num_body']


2. for "test" sample
    

In [ ]:
# path of a skeleton
path_skeleton="dataset/NTU_RGB+D/raw/S001C001P001R001A001.skeleton"
skeleton_name="S001C001P001R001A001"

3. Get the big .csv where you will get the right data after

In [ ]:
from utils.NTU_RGB.utils_dataset import summary_csv_NTU

df,path=summary_csv_NTU() #*  really slow operation

4.0 The dataframe where we will get the most important information

In [ ]:
from utils.NTU_RGB.utils_dataset import preprocess_csv_RGB_to_skeletondf
seq_len=30
out_len=30
df,path=preprocess_csv_RGB_to_skeletondf(seq_len=seq_len,out_len=out_len)

4. Get a sample of data that will be used inside NTU_RGB ( it is  data_set.liste_path)

In [ ]:
from utils.NTU_RGB.utils_dataset import data_rentrer_dans_DATASET_NTU
seq_len=32
out_len=64
df=data_rentrer_dans_DATASET_NTU(seq_len=seq_len,out_len=out_len)

get a sample of skeleton .npy

In [ ]:
path_skeleton="dataset/NTU_RGB+D/numpyed/S001C001P001R001A001.skeleton.npy"
import numpy as np
data = np.load(path_skeleton,allow_pickle=True).item()
# we want usually data['skel_body0'].shape

***

## Dataset and model runs (FED/Auto)

### Récupération du dataset et data_loader

- récupération du dataset et du dataloader de NTU_RGB (suit l'args précédement défini)

In [ ]:
from data_provider.data_factory import data_provider
args=Args()
data_set, data_loader=data_provider(args,flag="train") 

In [ ]:
entry=data_set.__getitem__(0)

In [ ]:
# Entry technical detail when you get back on data_set ._ Getitem __ (0)
"""
Entry is in the form:
(input_frames, nb_channels = 75)
"""

- Recovery of an item from data_loader

usual method

In [ ]:
(batch_x, batch_y, batch_x_mark, batch_y_mark)=enumerate(data_loader).__next__()[1]


alternative but faster method

In [ ]:
sample_name="S001C001P001R001A001"
entry=data_set.get_data_from_sample_name(name_skeleton=sample_name,num_body=0) #* Attention au numéro du body!
entry_batch=data_set.get_input_model(entry)



### Run of the model: Fed

- Setup of the model:

In [ ]:
from models.FEDformer import *
network=Model(args)
network.float()

-recuperation of a dataset entrance:

In [ ]:
# Technical detail of the input of the Data_loader model/
"""
It is in shape: batch_x, batch_y, batch_x_mark, batch_y_mark

and batch_x is in shape

(nb_batch  Input_len, nb_channel)
"""

In [ ]:
# Batch recovery
from data_provider.data_factory import data_provider
args=Args()
data_set, data_loader=data_provider(args,flag="train") 

(batch_x, batch_y, batch_x_mark, batch_y_mark)=enumerate(data_loader).__next__()[1]

- output of the Fedformers model (uses the front data)

In [ ]:
y=network(batch_x.float(), batch_x_mark.float(),None, batch_y_mark.float())

In [ ]:
# Technical detail of the Fedformers outing
""" 
y est de la forme: (batch_size, pred_len, nb_channel)
"""

### Run of exp_long_term_forecasting

Usual setings to take all at the top of the jupyter.

Model launch

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast
Exp=Exp_Long_Term_Forecast

exp = Exp(args)
exp.train(setting)

## Visualisation

In [ ]:
#Technical detail to use Plot_skeletons:
"""
To visualize a skeleton, the Skeleton array must be in shape: \
-[nb_joints, nb_dimension (3), nb_frames]

"""

example of input for plot skeletons

In [ ]:
from utils.NTU_RGB.utils_NTU import read_xyz
path_skeleton="dataset/NTU_RGB+D/raw/S001C001P001R001A001.skeleton"
skeleton = read_xyz(path_skeleton)
skeleton.shape

Plot a video

In [ ]:
# get a data path or a root path
import os
root_path='./dataset/NTU_RGB+D/'
data_path_npy=os.path.join(root_path,'numpyed/')
data_path_skel=os.path.join(root_path,'raw/')
skeleton="S001C001P001R001A001.skeleton"
path_skeleton=os.path.join(data_path_skel,skeleton)

print(path_skeleton)


In [ ]:
# parameters of the plot
num_body=0
# Check the number of body, if it is greater than 0 we outputs the skeleton num_body of the scene, else everything if num_body=-1
save_name="example"
path_folder_save="./videos/" # where we store the sample

In [ ]:
from utils.NTU_RGB.plot_skeleton import plot_skeleton
plot_skeleton(path_skeleton=path_skeleton,
              save_name=save_name,
              path_folder_save=path_folder_save,
              num_body=num_body)